# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [137]:
# Load environment variables
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [185]:
# Paths
import os
import sys


# Standard libraries
import pandas as pd
import numpy as np

# ML

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
#from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, log_loss, cohen_kappa_score, f1_score, root_mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
#from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
#from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor


In [139]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [140]:
fires_dt.head()

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


# Get X and Y

Create the features data frame and target data.

In [141]:
#Scaled variables
numeric_variable = ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']

#One-hot encoding variables
categorical_variable = ['month', 'day']

#Non-Sacalable variables; Coordinates should not be scaled for this project
numeric_ns_variable = ['coord_x', 'coord_y']



In [142]:
#All features = X (predictors)
feature_df = fires_dt.drop(columns="area")

#Target data = y (response)
target_df = fires_dt['area']

#Numeric scalable features
numeric_feature_df = fires_dt[numeric_variable]

#Numeric no scalable features
numeric_ns_feature_df = fires_dt[numeric_ns_variable]

#Categorical fetures
categorical_feature_df = fires_dt[categorical_variable]

#X
X= feature_df

#Y
Y = target_df


In [143]:


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [144]:
#Column Transform

column_transform_01 = ColumnTransformer(
    transformers=[
    ("numeric_scalable", StandardScaler(), numeric_variable),
    ("categorical", OneHotEncoder(handle_unknown="infrequent_if_exist"),categorical_variable)
], remainder= "passthrough"

)
preproc1 = column_transform_01

preproc1

ColumnTransformer(remainder='passthrough',
                  transformers=[('numeric_scalable', StandardScaler(),
                                 ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh',
                                  'wind', 'rain']),
                                ('categorical',
                                 OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                 ['month', 'day'])])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [145]:
pipe_num_yj = Pipeline([
    ('standardizer', StandardScaler()),
    ('transform', PowerTransformer(method='yeo-johnson'))
])


ctramsform_yj = ColumnTransformer(
    [
    ('numeric_yj', pipe_num_yj, numeric_variable),
    ('categorical', OneHotEncoder(handle_unknown="infrequent_if_exist"), categorical_variable)
    ], 
    remainder='passthrough'
)

preproc2 = ctramsform_yj
preproc2

ColumnTransformer(remainder='passthrough',
                  transformers=[('numeric_yj',
                                 Pipeline(steps=[('standardizer',
                                                  StandardScaler()),
                                                 ('transform',
                                                  PowerTransformer())]),
                                 ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh',
                                  'wind', 'rain']),
                                ('categorical',
                                 OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [146]:
#Regressors:
#Baseline: KNN-Regressor
#Advanced model: T

In [147]:
# Pipeline A = preproc1 + baseline
pipeline_a = Pipeline([
    ('preprocessing_1', preproc1),
    ("regressor_1",KNeighborsRegressor())  
])

pipeline_a

Pipeline(steps=[('preprocessing_1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric_scalable',
                                                  StandardScaler(),
                                                  ['ffmc', 'dmc', 'dc', 'isi',
                                                   'temp', 'rh', 'wind',
                                                   'rain']),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('regressor_1', KNeighborsRegressor())])

In [148]:
# Pipeline B = preproc2 + baseline
pipeline_b = Pipeline([
    ('preprocessing_2', preproc2),
    ("regressor_1",KNeighborsRegressor())
])

pipeline_b

Pipeline(steps=[('preprocessing_2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric_yj',
                                                  Pipeline(steps=[('standardizer',
                                                                   StandardScaler()),
                                                                  ('transform',
                                                                   PowerTransformer())]),
                                                  ['ffmc', 'dmc', 'dc', 'isi',
                                                   'temp', 'rh', 'wind',
                                                   'rain']),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('regressor_1', KNeighborsRegressor())])

In [149]:
# Pipeline C = preproc1 + advanced model
pipeline_c = Pipeline([
    ('preprocessing_1', preproc1),
    ("regressor_2",RandomForestRegressor(random_state=42)) 
])

pipeline_c

Pipeline(steps=[('preprocessing_1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric_scalable',
                                                  StandardScaler(),
                                                  ['ffmc', 'dmc', 'dc', 'isi',
                                                   'temp', 'rh', 'wind',
                                                   'rain']),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('regressor_2', RandomForestRegressor(random_state=42))])

In [150]:
# Pipeline D = preproc2 + advanced model
pipeline_d = Pipeline([
    ('preprocessing_2', preproc2),
    ("regressor_2",RandomForestRegressor(random_state=42))  
])

pipeline_d

Pipeline(steps=[('preprocessing_2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric_yj',
                                                  Pipeline(steps=[('standardizer',
                                                                   StandardScaler()),
                                                                  ('transform',
                                                                   PowerTransformer())]),
                                                  ['ffmc', 'dmc', 'dc', 'isi',
                                                   'temp', 'rh', 'wind',
                                                   'rain']),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('regressor_2', RandomForestRegressor(random_state=42))])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [151]:
scoring = {
    'RMSE': 'neg_root_mean_squared_error',
    'MAE': 'neg_mean_absolute_error',
    'R2': 'r2'
}



In [152]:
#param_grid knn
param_grid_knn = {
    'regressor_1__n_neighbors': range(1, 50, 2),  # Odd values to prevent ties
    'regressor_1__weights': ['uniform', 'distance'],  # Uniform (equal weight), Distance (inverse distance weighted)
    'regressor_1__metric': ['euclidean', 'manhattan', 'minkowski']  # Distance metrics
}



In [153]:
#param_grid random forest

param_grid_rf = {
    'regressor_2__n_estimators': [50, 100, 200, 300],  # Number of trees
    'regressor_2__max_depth': [None, 5, 10, 20],  # Tree depth
    'regressor_2__min_samples_split': [2, 5, 10],  # Minimum samples to split a node
    'regressor_2__min_samples_leaf': [1, 2, 4],  # Minimum samples per leaf
    'regressor_2__bootstrap': [True, False]  # Whether to sample with replacement
}


In [154]:
#Pipeline_A

grid_cv_a = GridSearchCV(
    estimator=pipeline_a, 
    param_grid=param_grid_knn, 
    scoring = scoring, 
    cv = 5,
    refit = "RMSE")

grid_cv_a.fit(X_train, Y_train)

c:\Users\jcp_2\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing_1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('numeric_scalable',
                                                                         StandardScaler(),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('categorical',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor_1', KNeighborsRegressor())]),
             param_grid={'regressor_1__metric': ['euclidean', 'manhattan',
                                                 'minkowski'],
                         'regressor_1__n_neighbors': range(1, 50, 2),
                         'regressor_1__weights': ['uniform', 'distance']},
             refit='RMSE',
             scoring={'MAE': 'neg_mean_absolute_error', 'R2': 'r2',
                      'RMSE': 'neg_root_mean_squared_error'})

In [155]:
#Pipeline_B

grid_cv_b = GridSearchCV(
    estimator=pipeline_b, 
    param_grid=param_grid_knn, 
    scoring = scoring, 
    cv = 5,
    refit = "RMSE")

grid_cv_b.fit(X_train, Y_train)

c:\Users\jcp_2\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing_2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('numeric_yj',
                                                                         Pipeline(steps=[('standardizer',
                                                                                          StandardScaler()),
                                                                                         ('transform',
                                                                                          PowerTransformer())]),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('categorical',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor_1', KNeighborsRegressor())]),
             param_grid={'regressor_1__metric': ['euclidean', 'manhattan',
                                                 'minkowski'],
                         'regressor_1__n_neighbors': range(1, 50, 2),
                         'regressor_1__weights': ['uniform', 'distance']},
             refit='RMSE',
             scoring={'MAE': 'neg_mean_absolute_error', 'R2': 'r2',
                      'RMSE': 'neg_root_mean_squared_error'})

In [157]:
#Pipeline_C

grid_cv_c = GridSearchCV(
    estimator=pipeline_c, 
    param_grid=param_grid_rf, 
    scoring = scoring, 
    cv = 5,
    refit = "RMSE",
    n_jobs=-1,  # Use all CPU cores
    verbose=2  # Show progress
    )

grid_cv_c.fit(X_train, Y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


c:\Users\jcp_2\anaconda3\envs\dsi_participant\lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\jcp_2\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing_1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('numeric_scalable',
                                                                         StandardScaler(),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('categorical',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor_2',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'regressor_2__bootstrap': [True, False],
                         'regressor_2__max_depth': [None, 5, 10, 20],
                         'regressor_2__min_samples_leaf': [1, 2, 4],
                         'regressor_2__min_samples_split': [2, 5, 10],
                         'regressor_2__n_estimators': [50, 100, 200, 300]},
             refit='RMSE',
             scoring={'MAE': 'neg_mean_absolute_error', 'R2': 'r2',
                      'RMSE': 'neg_root_mean_squared_error'},
             verbose=2)

In [160]:
#Pipeline_d

grid_cv_d = GridSearchCV(
    estimator=pipeline_d, 
    param_grid=param_grid_rf, 
    scoring = scoring, 
    cv = 5,
    refit = "RMSE",
    n_jobs=-1, # Use all CPU cores
    verbose=2  # Show progress
    )

grid_cv_d.fit(X_train, Y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


c:\Users\jcp_2\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing_2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('numeric_yj',
                                                                         Pipeline(steps=[('standardizer',
                                                                                          StandardScaler()),
                                                                                         ('transform',
                                                                                          PowerTransformer())]),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('categorical',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'da...
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'regressor_2__bootstrap': [True, False],
                         'regressor_2__max_depth': [None, 5, 10, 20],
                         'regressor_2__min_samples_leaf': [1, 2, 4],
                         'regressor_2__min_samples_split': [2, 5, 10],
                         'regressor_2__n_estimators': [50, 100, 200, 300]},
             refit='RMSE',
             scoring={'MAE': 'neg_mean_absolute_error', 'R2': 'r2',
                      'RMSE': 'neg_root_mean_squared_error'},
             verbose=2)

In [174]:
#Results
#pipe A
results_a = grid_cv_a.cv_results_
results_a = pd.DataFrame(results_a)
results_a.columns
results_a.sort_values('rank_test_RMSE').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor_1__metric,param_regressor_1__n_neighbors,param_regressor_1__weights,params,split0_test_RMSE,split1_test_RMSE,...,std_test_MAE,rank_test_MAE,split0_test_R2,split1_test_R2,split2_test_R2,split3_test_R2,split4_test_R2,mean_test_R2,std_test_R2,rank_test_R2
98,0.006519,0.000548,0.007812,0.000424,manhattan,49,uniform,"{'regressor_1__metric': 'manhattan', 'regresso...",-39.709672,-16.500962,...,3.732159,17,0.033381,-0.258631,-0.051246,-0.001426,0.065157,-0.042553,0.114738,1
96,0.006386,0.000657,0.008458,0.000893,manhattan,47,uniform,"{'regressor_1__metric': 'manhattan', 'regresso...",-39.655167,-16.731675,...,3.771344,25,0.036032,-0.294073,-0.054473,-0.000330,0.075080,-0.047553,0.130442,2
147,0.007634,0.000481,0.010915,0.000801,minkowski,47,distance,"{'regressor_1__metric': 'minkowski', 'regresso...",-40.036732,-17.272669,...,3.724167,3,0.017392,-0.379110,-0.058394,0.002615,0.119279,-0.059644,0.169637,10
47,0.007499,0.000899,0.010114,0.000586,euclidean,47,distance,"{'regressor_1__metric': 'euclidean', 'regresso...",-40.036732,-17.272669,...,3.724167,3,0.017392,-0.379110,-0.058394,0.002615,0.119279,-0.059644,0.169637,10
42,0.006233,0.000730,0.008482,0.001079,euclidean,43,uniform,"{'regressor_1__metric': 'euclidean', 'regresso...",-39.992905,-17.082049,...,3.759768,31,0.019542,-0.348838,-0.067308,0.003437,0.101998,-0.058234,0.154958,6


In [167]:
grid_cv_a.best_params_

{'regressor_1__metric': 'manhattan',
 'regressor_1__n_neighbors': 49,
 'regressor_1__weights': 'uniform'}

In [169]:
grid_cv_a.best_score_

-38.41954878713081

In [173]:
#Results
#pipe B
results_b = grid_cv_a.cv_results_
results_b = pd.DataFrame(results_a)
results_b.columns
results_b.sort_values('rank_test_RMSE').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor_1__metric,param_regressor_1__n_neighbors,param_regressor_1__weights,params,split0_test_RMSE,split1_test_RMSE,...,std_test_MAE,rank_test_MAE,split0_test_R2,split1_test_R2,split2_test_R2,split3_test_R2,split4_test_R2,mean_test_R2,std_test_R2,rank_test_R2
98,0.006519,0.000548,0.007812,0.000424,manhattan,49,uniform,"{'regressor_1__metric': 'manhattan', 'regresso...",-39.709672,-16.500962,...,3.732159,17,0.033381,-0.258631,-0.051246,-0.001426,0.065157,-0.042553,0.114738,1
96,0.006386,0.000657,0.008458,0.000893,manhattan,47,uniform,"{'regressor_1__metric': 'manhattan', 'regresso...",-39.655167,-16.731675,...,3.771344,25,0.036032,-0.294073,-0.054473,-0.000330,0.075080,-0.047553,0.130442,2
147,0.007634,0.000481,0.010915,0.000801,minkowski,47,distance,"{'regressor_1__metric': 'minkowski', 'regresso...",-40.036732,-17.272669,...,3.724167,3,0.017392,-0.379110,-0.058394,0.002615,0.119279,-0.059644,0.169637,10
47,0.007499,0.000899,0.010114,0.000586,euclidean,47,distance,"{'regressor_1__metric': 'euclidean', 'regresso...",-40.036732,-17.272669,...,3.724167,3,0.017392,-0.379110,-0.058394,0.002615,0.119279,-0.059644,0.169637,10
42,0.006233,0.000730,0.008482,0.001079,euclidean,43,uniform,"{'regressor_1__metric': 'euclidean', 'regresso...",-39.992905,-17.082049,...,3.759768,31,0.019542,-0.348838,-0.067308,0.003437,0.101998,-0.058234,0.154958,6


In [171]:
grid_cv_b.best_params_

{'regressor_1__metric': 'euclidean',
 'regressor_1__n_neighbors': 47,
 'regressor_1__weights': 'uniform'}

In [172]:
grid_cv_b.best_score_

-38.273674747535495

In [175]:
#Results
#pipe C
results_c = grid_cv_a.cv_results_
results_c = pd.DataFrame(results_a)
results_c.columns
results_c.sort_values('rank_test_RMSE').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor_1__metric,param_regressor_1__n_neighbors,param_regressor_1__weights,params,split0_test_RMSE,split1_test_RMSE,...,std_test_MAE,rank_test_MAE,split0_test_R2,split1_test_R2,split2_test_R2,split3_test_R2,split4_test_R2,mean_test_R2,std_test_R2,rank_test_R2
98,0.006519,0.000548,0.007812,0.000424,manhattan,49,uniform,"{'regressor_1__metric': 'manhattan', 'regresso...",-39.709672,-16.500962,...,3.732159,17,0.033381,-0.258631,-0.051246,-0.001426,0.065157,-0.042553,0.114738,1
96,0.006386,0.000657,0.008458,0.000893,manhattan,47,uniform,"{'regressor_1__metric': 'manhattan', 'regresso...",-39.655167,-16.731675,...,3.771344,25,0.036032,-0.294073,-0.054473,-0.000330,0.075080,-0.047553,0.130442,2
147,0.007634,0.000481,0.010915,0.000801,minkowski,47,distance,"{'regressor_1__metric': 'minkowski', 'regresso...",-40.036732,-17.272669,...,3.724167,3,0.017392,-0.379110,-0.058394,0.002615,0.119279,-0.059644,0.169637,10
47,0.007499,0.000899,0.010114,0.000586,euclidean,47,distance,"{'regressor_1__metric': 'euclidean', 'regresso...",-40.036732,-17.272669,...,3.724167,3,0.017392,-0.379110,-0.058394,0.002615,0.119279,-0.059644,0.169637,10
42,0.006233,0.000730,0.008482,0.001079,euclidean,43,uniform,"{'regressor_1__metric': 'euclidean', 'regresso...",-39.992905,-17.082049,...,3.759768,31,0.019542,-0.348838,-0.067308,0.003437,0.101998,-0.058234,0.154958,6


In [176]:
grid_cv_c.best_params_

{'regressor_2__bootstrap': True,
 'regressor_2__max_depth': 5,
 'regressor_2__min_samples_leaf': 4,
 'regressor_2__min_samples_split': 10,
 'regressor_2__n_estimators': 300}

In [177]:
grid_cv_c.best_score_

-41.61458230128819

In [178]:
#Results
#pipe C
results_d = grid_cv_a.cv_results_
results_d = pd.DataFrame(results_a)
results_d.columns
results_d.sort_values('rank_test_RMSE').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor_1__metric,param_regressor_1__n_neighbors,param_regressor_1__weights,params,split0_test_RMSE,split1_test_RMSE,...,std_test_MAE,rank_test_MAE,split0_test_R2,split1_test_R2,split2_test_R2,split3_test_R2,split4_test_R2,mean_test_R2,std_test_R2,rank_test_R2
98,0.006519,0.000548,0.007812,0.000424,manhattan,49,uniform,"{'regressor_1__metric': 'manhattan', 'regresso...",-39.709672,-16.500962,...,3.732159,17,0.033381,-0.258631,-0.051246,-0.001426,0.065157,-0.042553,0.114738,1
96,0.006386,0.000657,0.008458,0.000893,manhattan,47,uniform,"{'regressor_1__metric': 'manhattan', 'regresso...",-39.655167,-16.731675,...,3.771344,25,0.036032,-0.294073,-0.054473,-0.000330,0.075080,-0.047553,0.130442,2
147,0.007634,0.000481,0.010915,0.000801,minkowski,47,distance,"{'regressor_1__metric': 'minkowski', 'regresso...",-40.036732,-17.272669,...,3.724167,3,0.017392,-0.379110,-0.058394,0.002615,0.119279,-0.059644,0.169637,10
47,0.007499,0.000899,0.010114,0.000586,euclidean,47,distance,"{'regressor_1__metric': 'euclidean', 'regresso...",-40.036732,-17.272669,...,3.724167,3,0.017392,-0.379110,-0.058394,0.002615,0.119279,-0.059644,0.169637,10
42,0.006233,0.000730,0.008482,0.001079,euclidean,43,uniform,"{'regressor_1__metric': 'euclidean', 'regresso...",-39.992905,-17.082049,...,3.759768,31,0.019542,-0.348838,-0.067308,0.003437,0.101998,-0.058234,0.154958,6


In [179]:
grid_cv_d.best_params_

{'regressor_2__bootstrap': True,
 'regressor_2__max_depth': 5,
 'regressor_2__min_samples_leaf': 4,
 'regressor_2__min_samples_split': 10,
 'regressor_2__n_estimators': 300}

In [192]:
grid_cv_d.best_estimator_

Pipeline(steps=[('preprocessing_2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric_yj',
                                                  Pipeline(steps=[('standardizer',
                                                                   StandardScaler()),
                                                                  ('transform',
                                                                   PowerTransformer())]),
                                                  ['ffmc', 'dmc', 'dc', 'isi',
                                                   'temp', 'rh', 'wind',
                                                   'rain']),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('regressor_2',
                 RandomForestRegressor(max_depth=5, min_samples_leaf=4,
                                       min_samples_split=10, n_estimators=300,
                                       random_state=42))])

In [181]:
print("Best RMSE (Pipeline A):", -grid_cv_a.best_score_)
print("Best Params (Pipeline A):", grid_cv_a.best_params_)

print("Best RMSE (Pipeline B):", -grid_cv_b.best_score_)
print("Best Params (Pipeline B):", grid_cv_b.best_params_)

print("Best RMSE (Pipeline C):", -grid_cv_c.best_score_)
print("Best Params (Pipeline C):", grid_cv_c.best_params_)

print("Best RMSE (Pipeline D):", -grid_cv_d.best_score_)
print("Best Params (Pipeline D):", grid_cv_d.best_params_)


Best RMSE (Pipeline A): 38.41954878713081
Best Params (Pipeline A): {'regressor_1__metric': 'manhattan', 'regressor_1__n_neighbors': 49, 'regressor_1__weights': 'uniform'}
Best RMSE (Pipeline B): 38.273674747535495
Best Params (Pipeline B): {'regressor_1__metric': 'euclidean', 'regressor_1__n_neighbors': 47, 'regressor_1__weights': 'uniform'}
Best RMSE (Pipeline C): 41.61458230128819
Best Params (Pipeline C): {'regressor_2__bootstrap': True, 'regressor_2__max_depth': 5, 'regressor_2__min_samples_leaf': 4, 'regressor_2__min_samples_split': 10, 'regressor_2__n_estimators': 300}
Best RMSE (Pipeline D): 41.60388223885863
Best Params (Pipeline D): {'regressor_2__bootstrap': True, 'regressor_2__max_depth': 5, 'regressor_2__min_samples_leaf': 4, 'regressor_2__min_samples_split': 10, 'regressor_2__n_estimators': 300}


# Evaluate

+ Which model has the best performance?

In [191]:
# Pipeline A
best_model_a = grid_cv_a.best_estimator_
y_pred_a = best_model_a.predict(X_test)
rmse_a = mean_squared_error(Y_test, y_pred_a, squared=False)
print("Test RMSE (Pipeline A):", rmse_a)

# Pipeline B
best_model_b = grid_cv_b.best_estimator_
y_pred_b = best_model_b.predict(X_test)
rmse_b = mean_squared_error(Y_test, y_pred_b, squared=False)
print("Test RMSE (Pipeline B):", rmse_b)

# Pipeline C
best_model_c = grid_cv_c.best_estimator_
y_pred_c = best_model_c.predict(X_test)
rmse_c = mean_squared_error(Y_test, y_pred_c, squared=False)
print("Test RMSE (Pipeline C):", rmse_c)

# Pipeline D
best_model_d = grid_cv_d.best_estimator_
y_pred_d = best_model_d.predict(X_test)
rmse_d = mean_squared_error(Y_test, y_pred_d, squared=False)
print("Test RMSE (Pipeline D):", rmse_d)



Test RMSE (Pipeline A): 108.82041785558646
Test RMSE (Pipeline B): 108.76293885624393
Test RMSE (Pipeline C): 108.39126688542855
Test RMSE (Pipeline D): 108.38386066818136


c:\Users\jcp_2\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\jcp_2\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\jcp_2\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\jcp_2\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in versio

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.